# Part I: Introductory Material

## Chapter 2: The Basics

In [ ]:
int i1 = 1;
int i2{1};
auto i3{1.2};

cout << i1 << ' ' << i2 << ' ' << i3 << endl;

In [ ]:
double sum(const vector<double>&v) { 
    double s = 0;
    for (double x : v) {
        s += x;
    }
    return s;
}

In [ ]:
constexpr double square(double x) { return x * x; }

 `const` : value can not be changed.
 

 `constexpr`: to be valued at compile time.

In [ ]:
const int a = 1;
int b = 2;
constexpr double c1 = 1.4 * a;
// constexpr double c2 = 1.4 * b; // error

vector<double> v {1.2, 3.4, 4.5};
const double s1 = sum(v);
cout << s1 << endl;

// constexpr double s2 = sum(v); // error
constexpr double s3 = square(a);


`*`:
* in declaration: pointer.
* otherwise: contents of.

`&`: 
* in declaration: reference to.
* otherwise: address of.

`.`: access `struct` members through a name or reference.

`->`: access `sturct` members through a pointer.


In [ ]:
enum class Color { red, blue, green };

Color col = Color::red;


In [ ]:
namespace MyCode {
    class complex { };
    complex sqrt(complex);
    int main();
}

In [ ]:
static_assert(4 <= sizeof(int), "intergers are too small"); // check at compile time

## Chapter 3: Abstraction Mechanisms

`std::initializer_list`: define the initializer-list constructor

In [ ]:
Vector::Vector<std:initializer_list<double> lst):elem{new double[lst.size()], sz{lst.size()}} {
    copy(lst.begin(), lst.end(), elem);
}

In [ ]:
class Point
{
    private:
        int x, y;
    public:
        Point() : x(0), y(0) {}
        Point(int x, int y) : x(x), y(y) {}
}

In [ ]:
class Shape {
public:
    virtual Point center() const=0;
    virtual void move(Point to)=0;
    
    virtual void draw() const=0;
    virtual void rotate(int angle)=0;
    
    virtual ~Shape(){}
    
};

In [ ]:
void rotate_all(vector<Shape*>& v, int angle) 
{
    for (auto p : v)
        p->rotate(angle);
}

In [ ]:
class Circle : public Shape {
public:
    Circle(Point p, int rr);
    
    Point center() const { return x; }
    void move(Point to) { x = to; }
    
    void draw() const;
    void rotate(int) {}
    
private:
    Point x;
    int r;
};

In [ ]:
class Smiley : public Circle {
public:
    Smiley(Point p, int r): Circle{p, r}, mouth{ nullptr } {}
    
    ~Smiley()
    {
        delete mouth;
        for (auto p : eyes) delete p;
    }
    
    void move(Point to);
    
    void draw() const;
    void rotate(int);
    
    void add_eye(Shape* s) { eyes.push_back(s); }
    void set_mouth(Shape* s);
    virtual void wink(int i);
    
private:
    vector<Shape*> eyes;
    Shape* mouth;
}

void Smiley::draw()
{
    Circle::draw();
    for (auto p : eyes)
        p->draw();
    mouth->draw();
}


* copy constructor: called when class was created.
* copy assignment：called when assigned. 

In [ ]:
class A {
 public:
  A() {
    std::cout << "A() called!\n";
  }
  A(const A& a) {
    std::cout << "A(const A& a) called!\n";
  }
  A& operator=(const A& a) {
    std::cout << "operator=(const A& a)!\n";
    return *this;
  }
};


In [ ]:
 A a1;
 A a2 = a1;
 A a3(a1);
 a3 = a2;

In [ ]:
// function objects
template<typename T>
class LessTan {
    const T val;
public:
    LessTan(const T& v):val(v) {}
    bool operator()(const T& x) const { return x < val; }
};

In [ ]:
LessTan<int> lti{42};
cout << lti(10);

##  Chapter 4: Containers and Algorithms

In [ ]:
// I/O
struct Entry {
    string name;
    int number;
};


In [ ]:
int sum(){ return 1;}

In [ ]:
ostream& operator<<(ostream& os, const Entry& e) 
{
    return os << e.name << ' ' << e.number << endl;
}

In [2]:
#include <iostream>
#include <algorithm>
#include <vector>
using namespace std;

* `unique_copy`

In [ ]:
// back_insert, unique_copy
vector<int>a{1, 2, 3, 2, 3, 4, 5};
sort(a.begin(), a.end());
vector<int>b(10);
unique_copy(a.begin(), a.end(), b.begin());


* `find`

In [3]:
vector<int>a{1, 2, 3, 4, 5};
auto p = find(a.begin(), a.end(), 2);
cout << p - a.begin() << endl;

1


In [5]:
template<typename C, typename V>
vector<typename C::iterator> find_all(C& c, V v) {
    vector<typename C::iterator> res;
    for (auto p = c.begin(); p != c.end(); ++p) {
        if (*p == v)
            res.push_back(p);
    }
    return res;
}
auto ps = find_all(a, 2);

In [8]:
template<typename T>
using Iterator = typename T::iterator;

template<typename C, typename V>
vector<Iterator<C>> find_all(C& c, V v) {
    vector<Iterator<C>> res;
    for (auto p = c.begin(); p != c.end(); ++p) {
        if (*p == v)
            res.push_back(p);
    }
    return res;
}
auto ps = find_all(a, 2);

* `unique_ptr`: to represent unique ownership
* `shared_ptr`: to represent shared ownership 

## Chapter 5: Concurrency and Utilities

* `join`: wait for the thread to terminate. 

In [ ]:
mutex m; // controlling mutex
int sh; // shared data
void f()
{
    unique_lock<mutex> lck {m}; // acquire mutex
    sh += 7; // manipulate shared data
}

In [ ]:
// avoid dead lock
void f() {
    unique_lock<mutex> lck1 {m1, defer_lock};
    unique_lock<mutex> lck2 {m2, defer_lock};
    unique_lock<mutex> lck3 {m3, defer_lock};
    
    lock(lck1, lck2, lck3);
}

In [ ]:
class Message {
    
};
queue<Message> mqueue;
condition_variable mcond;
mutex mmutex;

void consumer() {
    while (true) {
        unique_lock<mutex> lck{mmutex};
        while (mcond.wait(lck));
        auto m = mqueue.front();
        mqueue.pop();
        lck.unlock();
    }
}

void producer() {
    while (true) {
        Message m;
        unique_lock<mutex> lck {mmutex};
        mqueue.push(m);
        mcond.notify_one();
    }
}

* `future`, `promise`: return a value from a task spawned on a separated thread 
* `packaged_task`: help launch tasks and connect up the mechanisms for returning a result
* `async`: launch of a task in a manner very similar to calling a function

In [ ]:
void f(promise<X>& px) // a task: place the result in px
{
    // ...
    try {
        X res;
        // ... compute a value for res ...
        px.set_value(res);
    }
    catch (...) { // oops: couldn’t compute res
        // pass the exception to the future’s thread:
        px.set_exception(current_exception());
    }
}
void g(future<X>& fx) // a task: get the result from fx
{
    // ...
    try {
        X v = fx.g et(); // if necessary, wait for the value to get computed
        // ... use v ...
        }
    catch (...) { // oops: someone couldn’t compute v
        // ... handle error ...
    }
}

In [ ]:
double accum(double∗ beg, double ∗ end, double init)
// compute the sum of [beg:end) starting with the initial value init
{
    return accumulate(beg,end,init);
}

double comp2(vector<double>& v)
{
    using Task_type = double(double∗,double∗,double); // type of task
    
    packaged_task<Task_type> pt0 {accum}; // package the task (i.e., accum)
    packaged_task<Task_type> pt1 {accum};
    
    future<double> f0 {pt0.get_future()}; // get hold of pt0’s future
    future<double> f1 {pt1.get_future()}; // get hold of pt1’s future
    
    double∗ first = &v[0];
    thread t1 {move(pt0),first,first+v.siz e()/2,0}; // star t a thread for pt0
    thread t2 {move(pt1),first+v.siz e()/2,first+v.siz e(),0}; // star t a thread for pt1
    // ...
    return f0.get() + f1.get();
}

In [ ]:
double comp4(vector<double>& v)
// spawn many tasks if v is large enough
{
    if (v.siz e()<10000) return accum(v.begin(),v.end(),0.0);
    auto v0 = &v[0];
    auto sz = v.siz e();
    auto f0 = async(accum,v0,v0+sz/4,0.0); // first quarter
    auto f1 = async(accum,v0+sz/4,v0+sz/2,0.0); // second quarter
    auto f2 = async(accum,v0+sz/2,v0+sz∗3/4,0.0); // third quarter
    auto f3 = async(accum,v0+sz∗3/4,v0+sz,0.0); // four th quar ter
    return f0.get()+f1.g et()+f2.get()+f3.get(); // collect and combine the results
}   

In [ ]:
* `decltype()`: returns the declared type of its argument. 

In [ ]:
void f(valarray<double>& a1, valarray<double>& a2) {
    valarray<double> a = a1 * 3.14 + a2 / a1;
    a2 += a1 * 3.14;
    a = abs(a);
    double d = a2[7];
}

In [9]:
#include <limits>

In [11]:
cout << numeric_limits<int>::max();

2147483647

# Part II: Basic Facilities

## Chapter 6: Types and Declarations


In [2]:
bool b1{ 1 == 2 };

// non-null pointer converts to true,
// pointers with value nullptr converts to false.
int *p;
bool b = p;
bool b2 { p != nullptr };

* `wchar_t`: Hold a larger character set, such as Unicode
* `char16_t`: Hold 16-bit character set, such as UTF-16 
* `char32_t`: Hold 32-bit character set, such as UTF-32 

* Octal: '\60', 060
* Hexadecimal: '\x30', 0x30
* Decimal: 48

In [10]:
#include<iostream>
auto ac = alignof('c');

input_line_22:2:12: warning: 'alignof' applied to an expression is a GNU extension [-Wgnu-alignof-expression]
 auto ac = alignof('c');
           ^
input_line_22:3:14: error: use of undeclared identifier 'a'
alignas(int) a;
             ^


Interpreter Error: 

In [ ]:
double sqrt(double); // function declaration
extern int error_number; // variable declaration
struct User; // type name declaration

In [ ]:
char* kings[]; // array of pointers to char
char(*kings)[]; // pointer to an arrar of char    

In [ ]:
// prefer = when using auto.
auto z1 {99}; // z1 is an initializer_list<int>
auto z2 = 99; // z2 is an int

vector<int> v1 {99}; // v1 is a vector of 1 element with the value 99
vector<int> v2(99); // v2 is a vector of 99 elements each with the default value 0

char buf[max] {}; // initialize every char to 0

In [ ]:
template<class T, class U>
auto operator+(const Matrix<T>& a, const Matrix<U>& b) −> Matrix<decltype(T{}+U{})>
{
    Matrix<decltype(T{}+U{})> res;
    for (int i=0; i!=a.rows(); ++i)
        for (int j=0; j!=a.cols(); ++j)
            res(i,j) += a(i,j) + b(i,j);
    
    return res;
}

In [ ]:
# Type Aliases
using Pchar = char∗; // pointer to character
using PF = int(∗)(double); // pointer to function taking a double and returning an int

# Chapter 7: Pointers, Arrays, and References